<a href="https://colab.research.google.com/github/linhkid/gdg-codelab-25/blob/main/GDG_Gemma2.0_multiagent_funccall_vertex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Codelab: Build your first agentic AI with Gemma 2.0 and Vertex AI

# Step 1: Setup and Authentication

In [44]:
# Install dependencies and authenticate with Vertex AI

# @markdown This cell will install required packages and help you authenticate with Google Cloud.

!pip install -q -U google-cloud-aiplatform
!pip install -q matplotlib pandas numpy

from google.colab import auth
import os
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown

# Authenticate to Google Cloud
auth.authenticate_user()

print("✅ Required packages installed. Authenticating with Vertex AI...\n")

✅ Required packages installed. Authenticating with Vertex AI...



In [86]:
#MODEL_NAME = "projects/991182027087/locations/asia-southeast1/models/gemma-2-2b-it-1742823574849"
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632"

# Step 2: Initialize Vertex AI for Gemma 2.0

In [106]:
# Initialize Vertex AI and Gemma 2.0
# @markdown Configure your project and set up Gemma 2.0 via Vertex AI

import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel

# Set your project ID
PROJECT_ID = "gdg-codelab-12thMay "  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    try:
        # Try to retrieve project ID from environment variable
        PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
        print(f"Using project ID from environment: {PROJECT_ID}")
    except:
        print("❌ Please set your Google Cloud project ID")

# Set location
LOCATION = "asia-southeast1"  # Vertex AI region

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Gemma 2.0 model setup
MODEL_NAME = "projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632"  # Instruction-tuned Gemma 2.0 model

try:
    # Load the Gemma 2.0 model from Vertex AI
    model = GenerativeModel(MODEL_NAME)
    print(f"✅ Successfully initialized {MODEL_NAME} on Vertex AI")
except Exception as e:
    print(f"❌ Error initializing model: {e}")
    print("Please check your project configuration and model availability in Vertex AI")

# Helper function for generating responses

def generate_response(prompt, temperature=0.2, max_output_tokens=1024, top_p=0.8):
    """Generate a response from Gemma 2.0 on Vertex AI"""
    request_json = {
        "instances": [
            {
                "inputs": prompt
            }
        ]
    }
    try:
        generation_config = GenerationConfig(
            temperature=temperature,
            max_output_tokens=max_output_tokens,
            top_p=top_p
        )

        response = model.generate_content(
            json.dumps(request_json),
            generation_config=generation_config
        )

        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response."

# Helper function for displaying markdown format

def display_markdown(text, render_markdown=True):
    """
    Display text as Markdown in a Jupyter notebook.

    Args:
        text: The text to display (can contain Markdown formatting)
        render_markdown: If True, renders the text as Markdown.
                        If False, displays the raw Markdown source in a code block.

    Returns:
        None: Displays the formatted content in the notebook
    """
    from IPython.display import display, Markdown, HTML

    if render_markdown:
        # Display text with Markdown rendering
        display(Markdown(text))
    else:
        # Display raw Markdown source code in a code block
        display(Markdown(f"```markdown\n{text}\n```"))


✅ Successfully initialized projects/991182027087/locations/asia-southeast1/endpoints/5762905479034437632 on Vertex AI


## Application 1: Structured Information Extraction

In [107]:
# Extract structured data from text using Gemma 2.0 on Vertex AI

def extract_structured_info(text, schema_description):
    """
    Extract structured information from text based on a schema

    Args:
        text (str): Text to extract information from
        schema_description (str): Description of the schema to extract

    Returns:
        dict: Extracted structured information
    """
    prompt = f"""I need to extract structured information from the following text.

    Text: "{text}"

    Please extract the following information:
    {schema_description}

    Return your answer as a markdown bullet points.
    """
    response = generate_response(prompt, temperature=0.1)

    # Extract JSON from response
    return response

# Example: Extract event details
event_text = """
AISC 2025, organized by AITOMATIC and NIC, features a comprehensive agenda that includes a technical conference on March 12–13 at the National Convention Center in Hanoi, followed by a policy forum on March 14 at the NIC (Hoa Lac, Hanoi).
Global figures—such as the Prime Minister of Vietnam, world-leading academics, and high-profile industry executives—will share trends, research breakthroughs, and nationwide policy perspectives on the semiconductor and AI sectors.
Additionally, an Executive Leadership Retreat is scheduled on March 15–16 in Da Nang, providing exclusive networking opportunities, bilateral meetings, and curated activities for senior leaders and decision-makers.

Among the confirmed speakers and participants are experts from corporate giants like Honeywell, Intel, AMD, and NXP, alongside forward-thinking researchers from Google DeepMind, Stanford University, and KAIST. Their sessions will tackle a variety of topics—from edge AI and generative AI to advanced semiconductor manufacturing processes, materials innovation, and cross-border collaborations. Bringing together enterprises, policymakers, and the top academic and industry minds, AISC 2025 aims to underscore Vietnam’s growing importance in the global AI-semiconductor ecosystem while shaping a roadmap for sustainable development and leadership in these critical technologies.
Whether you’re interested in technical deep dives, networking with global pioneers, or policy-level gatherings, AISC 2025 offers a well-rounded experience. Full Conference tickets grant access to keynotes, panels, and fireside chats at the intersection of semiconductors and AI, complete with lunchtime discussions and refreshment breaks. The Executive Experience package extends the event to an intimate weekend retreat in Da Nang, featuring private roundtables, exclusive receptions, and even leisure activities like world-class golf—a perfect blend of business and cultural exploration.
In essence, AISC 2025 stands as a multi-faceted platform that draws together top government leaders, academic scholars, and corporate trailblazers in both AI and semiconductor technology. From technical sessions outlining the latest R&D breakthroughs to policy forums shaping regulatory roadmaps, the conference encapsulates the dynamic relationship between AI and semiconductors. Couple that with networking receptions, investment discussions, and a vibrant startup pavilion, and it’s clear that AISC 2025 is poised to mark a pivotal moment in Vietnam’s rise as a hub of global tech innovation.
"""

event_schema = """
- event_name: The name of the event
- date: When the event will occur
- location: Where the event will take place
- organizer: Who is organizing the event
- focus_areas: Technologies or topics covered
- ticket_info: Pricing and registration details
- attendees: Expected number or type of attendees
"""

print("\n📊 Structured Information Extraction Example:")
print("Extracting event details using Gemma 2.0 on Vertex AI...\n")

event_details = extract_structured_info(event_text, event_schema)

print("Extracted Event Details:")
#print(json.dumps(event_details, indent=2))
print(display_markdown(event_details))


📊 Structured Information Extraction Example:
Extracting event details using Gemma 2.0 on Vertex AI...

Extracted Event Details:


Here's the extracted information in markdown bullet points:

- **event_name:** AISC 2025
- **date:** March 12-13, March 14, March 15-16, 2025
- **location:** National Convention Center in Hanoi, Vietnam; Da Nang, Vietnam
- **organizer:** AITOMATIC and NIC
- **focus_areas:** 
    - Semiconductor and AI sectors
    - Edge AI
    - Generative AI
    - Advanced semiconductor manufacturing processes
    - Materials innovation
    - Cross-border collaborations
- **ticket_info:** 
    - Full Conference tickets grant access to keynotes, panels, fireside chats, lunchtime discussions, and refreshment breaks.
    - Executive Experience package includes an intimate weekend retreat with private roundtables, exclusive receptions, and leisure activities like world-class golf.
- **attendees:** 
    - Global figures (Prime Minister of Vietnam, world-leading academics, high-profile industry executives)
    - Experts from corporate giants (Honeywell, Intel, AMD, NXP)
    - Researchers from Google DeepMind, Stanford University, and KAIST
    - Senior leaders and decision-makers 
    - Startups 
    - Policymakers 
    - Government leaders 
    - Academic scholars 
    - Corporate trailblazers in AI and semiconductor technology 




None


## Application 2: Multi-agent Research System powered by Gemma 2.0

In [108]:
# @markdown Build a simple multi-agent research system using Gemma 2.0 on Vertex AI

class ResearchAgent:
    """
    A multi-agent system for research powered by Gemma 2.0 on Vertex AI
    """

    def __init__(self, model, debug=False):
        self.model = model
        self.debug = debug
        self.agents = {
            "planner": self._planning_agent,
            "researcher": self._research_agent,
            "analyzer": self._analysis_agent,
            "reporter": self._report_agent
        }

    def _planning_agent(self, query):
        """Plan the research strategy"""
        prompt = f"""You are a research planning specialist.

Given the research query: "{query}"

Create a detailed research plan with the following components:
1. Key questions to answer
2. Necessary data points to collect
3. Analysis methods to apply
4. Structure for the final report

Return a structured, step-by-step plan.
"""
        return generate_response(prompt, temperature=0.2)

    def _research_agent(self, query, plan):
        """Gather information based on the research plan"""
        prompt = f"""You are a thorough research specialist.

Research query: "{query}"

Research plan:
{plan}

Simulate the research data collection process and provide:
1. Key facts and data points you would gather
2. Sources you would consult (simulated)
3. Any challenges in data collection
4. Preliminary observations from the collected data

Present this as a research notes document.
"""
        return generate_response(prompt, temperature=0.3)

    def _analysis_agent(self, research_notes):
        """Analyze the gathered information"""
        prompt = f"""You are a data analysis specialist.

Research notes:
{research_notes}

Provide a comprehensive analysis including:
1. Key patterns and trends
2. Notable correlations
3. Potential causalities
4. Gaps requiring further research
5. Statistical observations (simulated)

Present this as an analytical summary.
"""
        return generate_response(prompt, temperature=0.2)

    def _report_agent(self, query, plan, research_notes, analysis):
        """Generate a final research report"""
        prompt = f"""You are a professional report writer.

Original query: "{query}"

Research plan:
{plan}

Research notes:
{research_notes[:500]}... [notes truncated]

Analysis:
{analysis[:500]}... [analysis truncated]

Create a well-structured, professional research report with:
1. Executive summary
2. Introduction and background
3. Methodology
4. Key findings
5. Discussion of implications
6. Conclusions and recommendations
7. Appendix (simulated references)

The report should be comprehensive yet concise.
"""
        return generate_response(prompt, temperature=0.2, max_output_tokens=2048)

    def execute_research(self, query):
        """Execute the full research pipeline"""
        if self.debug:
            print("📋 Starting research process...")

        # Step 1: Planning
        if self.debug:
            print("🧩 Planning research approach...")
        plan = self.agents["planner"](query)
        if self.debug:
            print("✅ Research plan created")

        # Step 2: Research
        if self.debug:
            print("🔍 Gathering research data...")
        research_notes = self.agents["researcher"](query, plan)
        if self.debug:
            print("✅ Research data collected")

        # Step 3: Analysis
        if self.debug:
            print("📊 Analyzing research data...")
        analysis = self.agents["analyzer"](research_notes)
        if self.debug:
            print("✅ Analysis complete")

        # Step 4: Reporting
        if self.debug:
            print("📝 Generating final report...")
        report = self.agents["report_agent"](query, plan, research_notes, analysis)
        if self.debug:
            print("✅ Report generated")

        # Return the complete research package
        return {
            "query": query,
            "plan": plan,
            "research_notes": research_notes,
            "analysis": analysis,
            "report": report
        }

# Create a research agent
research_system = ResearchAgent(model, debug=True)

# Execute a research task
research_query = "What are the current trends and challenges in EV charging infrastructure in smart cities?"

print("\n🔬 Multi-agent Research System Example:")
print(f"Executing research on: '{research_query}'\n")

research_results = research_system.execute_research(research_query)

# Display the final report
print("\n📑 Final Research Report:")
display(Markdown(research_results["report"]))


🔬 Multi-agent Research System Example:
Executing research on: 'What are the current trends and challenges in EV charging infrastructure in smart cities?'

📋 Starting research process...
🧩 Planning research approach...
✅ Research plan created
🔍 Gathering research data...
Error generating response: 'str' object has no attribute 'get'
✅ Research data collected
📊 Analyzing research data...
✅ Analysis complete
📝 Generating final report...


KeyError: 'report_agent'